In [2]:
#모듈 import

from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras import regularizers

import os
import numpy as np
import tensorflow as tf
import keras

In [3]:
#랜덤 시드 고정

np.random.seed(42)
tf.random.set_seed(42)

#데이터셋 불러오기

(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
print(X_train.shape, X_test.shape)

#데이터 정규화

X_train = X_train / 255.
X_test = X_test / 255.

(60000, 28, 28) (10000, 28, 28)


In [4]:
#레이블의 개수와 형태 확인

np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [13]:
#신경망 구축
#신경망 구축에서 dropdout 적용
#가중치 감소 적용(L2, L1)
#가중치 초기화 적용(He_normal)
#Batch Noramlization 적용

He_normal = keras.initializers.he_normal(seed=None)
batch_normal = keras.layers.BatchNormalization()

model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(64, activation='relu',
          kernel_regularizer=regularizers.l2(0.01),
          activity_regularizer=regularizers.l1(0.01),
          kernel_initializer = He_normal),
          batch_normal,
    Dropout(0.5),
    Dense(10, activation='softmax')
])

In [14]:
#compile
#compile 과정에서 학습률 감소 적용

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1 = 0.89)
             , loss='sparse_categorical_crossentropy'
             , metrics=['accuracy'])

In [15]:
#파라미터 저장 경로 설정

checkpoint_filepath = "FMbest.hdf5"

In [16]:
#조기 종료 옵션 설정
#loss 값이 10회동안 변화가 없으면 조기종료

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)

In [17]:
#종료 기준은 val_loss
#가장 좋은 결과만 저장
#가중치만 저장
#저장 단위는 에포크

save_best = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True,
    save_weights_only=True, mode='auto', save_freq='epoch', options=None)


In [18]:
#모델 학습

model.fit(X_train, y_train, batch_size=32, epochs=30, verbose=1, 
          validation_data=(X_test,y_test), 
          callbacks=[early_stop, save_best])

Epoch 1/30
1872/1875 [============================>.] - ETA: 0s - loss: 1.1778 - accuracy: 0.7365
Epoch 1: val_loss improved from inf to 0.88955, saving model to FMbest.hdf5
1875/1875 [==============================] - 6s 3ms/step - loss: 1.1774 - accuracy: 0.7365 - val_loss: 0.8896 - val_accuracy: 0.7643
Epoch 2/30
1856/1875 [============================>.] - ETA: 0s - loss: 0.8946 - accuracy: 0.7443
Epoch 2: val_loss improved from 0.88955 to 0.71985, saving model to FMbest.hdf5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.8946 - accuracy: 0.7443 - val_loss: 0.7198 - val_accuracy: 0.7916
Epoch 3/30
1861/1875 [============================>.] - ETA: 0s - loss: 0.8909 - accuracy: 0.7348
Epoch 3: val_loss did not improve from 0.71985
1875/1875 [==============================] - 5s 3ms/step - loss: 0.8914 - accuracy: 0.7347 - val_loss: 0.7979 - val_accuracy: 0.7484
Epoch 4/30
1862/1875 [============================>.] - ETA: 0s - loss: 0.9038 - accuracy: 0.7269
Epoch 

In [19]:
#조기 종료 직전의 모델 평가

model.predict(X_test[0:1])
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.7111 - accuracy: 0.7917 - 399ms/epoch - 1ms/step


In [22]:
#콜백으로 Best 모델 파라미터를 불러서 모델 예측

model.load_weights(checkpoint_filepath)
model.predict(X_test[0:1])

array([[8.2574870e-06, 2.4688487e-05, 9.0093181e-06, 1.0264771e-04,
        6.9738694e-06, 2.1217543e-01, 8.4622443e-06, 1.6989833e-01,
        2.9159128e-04, 6.1747456e-01]], dtype=float32)

In [21]:
#Best 모델 파라미터 평가

test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.6620 - accuracy: 0.8148 - 393ms/epoch - 1ms/step
